In [ ]:
from qiskit_dynamics import Solver, DynamicsBackend
from qiskit_dynamics.backend import default_experiment_result_function
from qiskit_dynamics.array import Array
import jax
from qiskit.providers.fake_provider import *
from qiskit import pulse

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
import numpy as np
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver

#gate_backend = provider.get_backend('simulator_statevector')

gate_backend =FakeManila()
gate_backend.configuration().hamiltonian['qub'] = {'0': 2,'1': 2,'2': 2,'3': 2,'4': 2}
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
Array.set_default_backend("jax")
pulse_backend = DynamicsBackend.from_backend(gate_backend, evaluation_mode="sparse")
solver_options = {"method": "jax_odeint", "atol": 1e-6, "rtol": 1e-8}
pulse_backend.set_options(solver_options=solver_options)
pulse_backend.configuration = lambda: gate_backend.configuration()
print("Porte logiche supportate dal backend ibm_nazca:", gate_backend.configuration().basis_gates)

In [ ]:
from qiskit_nature import settings
settings.use_pauli_sum_op = False

In [ ]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer,ActiveSpaceTransformer
import re
my_dict = {}

def get_qubit_op(dist):
    # Define Molecule
    ultra_simplified_ala_string = f"""
    H 0.0 0.0 0.0
    H 0.0 0.0 {dist}
    """
    
    driver = PySCFDriver(
        atom=ultra_simplified_ala_string.strip(),
        basis='sto3g',
        unit=DistanceUnit.ANGSTROM
    )
    qmolecule = driver.run()
    num_active_electrons = 2
    num_active_orbitals = 2
    as_transformer = ActiveSpaceTransformer(num_active_electrons,num_active_orbitals)
    problem = as_transformer.transform(qmolecule)
    mapper = ParityMapper(num_particles = problem.num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    '''num_active_electrons = 2
    num_active_orbitals = 2
    as_transformer = ActiveSpaceTransformer(num_active_electrons,num_active_orbitals)
    problem = as_transformer.transform(qmolecule)
    mapper = ParityMapper(num_particles = problem.num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    print(qubit_op)'''
    for pauli in qubit_op:
        #print(pauli)
        str_info = pauli.__str__()
        #print(f"Rappresentazione della stringa: {str_info}")
    
        # Estrai i coefficienti
        coeffs_start = str_info.find("coeffs=[") + len("coeffs=[")
        coeffs_end = str_info.find("]", coeffs_start)
        coeffs_str_complex = complex(str_info[coeffs_start:coeffs_end].strip())
        coeffs_str = coeffs_str_complex.real
        #print(coeffs_str)
        
        # Estrai gli operatori come stringa
        operators_start = str_info.find("[") + len("[")
        operators_end = str_info.find("])", operators_start)
        operators_str = str_info[operators_start:operators_end].strip()
        label_start = operators_str.find("'") + 1
        label_end = operators_str.find("'", label_start)
        label = operators_str[label_start:label_end]
        #print(label)
    
        '''# Utilizza espressioni regolari per estrarre la parte reale e immaginaria dei coefficienti
        coeffs_match = re.findall(r"(-?\d+\.\d+)([+-]\d+\.\d+j)?", coeffs_str)
    
        # Inizializza i coefficienti come float
        real_coeffs = [float(match[0]) for match in coeffs_match]
        imag_coeffs = [float(match[1]) if match[1] else 0.0 for match in coeffs_match]
    
        # Somma i coefficienti complessi
        complex_coeff = sum(complex(real, imag) for real, imag in zip(real_coeffs, imag_coeffs))'''
    
        # Aggiungi il coefficiente all'operatore nel dizionario
        #current_coeff = my_dict.get(operators_str, 0.0)
        #my_dict[operators_str] = coeffs_str
        my_dict[label] = coeffs_str
        
        
    return qubit_op, my_dict,problem,problem.nuclear_repulsion_energy

In [ ]:
from qiskit.pulse import Schedule, GaussianSquare, Drag, Delay, Play, ControlChannel, DriveChannel

def drag_pulse(backend, amp, angle):
  backend_defaults = backend.defaults()
  inst_sched_map = backend_defaults.instruction_schedule_map
  x_pulse = inst_sched_map.get('x', (0)).filter(channels = [DriveChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
  duration_parameter = x_pulse.parameters['duration']
  sigma_parameter = x_pulse.parameters['sigma']
  beta_parameter = x_pulse.parameters['beta']
  pulse1 = Drag(duration=duration_parameter, sigma=sigma_parameter, beta=beta_parameter, amp=amp, angle=angle)
  return pulse1

In [ ]:
def cr_pulse(backend, amp, angle, duration):
  backend_defaults = backend.defaults()
  inst_sched_map = backend_defaults.instruction_schedule_map
  cr_pulse = inst_sched_map.get('cx', (1, 0)).filter(channels = [ControlChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
  cr_params = {}
  cr_params['duration'] = cr_pulse.parameters['duration']
  cr_params['amp'] = cr_pulse.parameters['amp']
  cr_params['angle'] = cr_pulse.parameters['angle']
  cr_params['sigma'] = cr_pulse.parameters['sigma']
  cr_params['width'] = cr_pulse.parameters['width']
  cr_risefall = (cr_params['duration'] - cr_params['width']) / (2 * cr_params['sigma'])
  angle_parameter = angle
  duration_parameter =  duration
  sigma_parameter = cr_pulse.parameters['sigma']
  width_parameter = int(duration_parameter - 2 * cr_risefall * cr_params['sigma'])
  #declare pulse parameters and build GaussianSquare pulse
  pulse1 = GaussianSquare(duration = duration_parameter, amp = amp, angle = angle_parameter, sigma = sigma_parameter, width=width_parameter)
  return pulse1

In [ ]:
def sx_pulse(backend, amp, angle):
    backend_defaults = backend.defaults()
    inst_sched_map = backend_defaults.instruction_schedule_map
    h_pulse = h_pulse = inst_sched_map.get('sx', 1).filter(channels = [ControlChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
    duration_parameter = h_pulse.parameters['duration']
    sigma_parameter = h_pulse.parameters['sigma']
    beta_parameter = h_pulse.parameters['beta']
    pulse1 = Drag(duration=duration_parameter, sigma=sigma_parameter, beta=beta_parameter, amp=amp, angle=angle)
    return pulse1

In [ ]:
def HE_pulse(backend, amp, angle, width):
    with pulse.build(backend) as my_program1:
        # layer 1
        sched_list = []
        '''with pulse.build(backend) as sched1:
            qubits = (0, 1)
            for i in range(2):
                pulse.play(sx_pulse(backend, amp[i], angle[i]), DriveChannel(qubits[i]))
        sched_list.append(sched1)'''
        with pulse.build(backend) as sched1:
            qubits = (0, 1)
            for i in range(2):
                pulse.play(drag_pulse(backend, amp[i], angle[i]), DriveChannel(qubits[i]))
        sched_list.append(sched1)

        with pulse.build(backend) as sched2:
            uchan = pulse.control_channels(0, 1)[0]
            pulse.play(cr_pulse(backend, amp[0], angle[2], width[0]), uchan)
        sched_list.append(sched2)

        with pulse.build(backend) as sched4:
            qubits = (0, 1)
            for i in range(2):
                pulse.play(drag_pulse(backend, amp[i], angle[i]), DriveChannel(qubits[i]))
        sched_list.append(sched4)

        with pulse.build(backend) as my_program:
            with pulse.transpiler_settings(initial_layout=[0, 1]):
                with pulse.align_sequential():
                    for sched in sched_list:
                        pulse.call(sched)

    duration_in_cycles = my_program.duration
    dt = backend.configuration().dt
    duration_in_ns = duration_in_cycles * dt * 1e9  # conversione in nanosecondi

    print(f"Durata totale del programma di impulsi: {duration_in_ns} ns")
    
    return my_program

In [ ]:
import copy
import time
from scipy.optimize import minimize, LinearConstraint

def measurement_pauli(prepulse, pauli_string, backend, n_qubit):
    with pulse.build(backend) as pulse_measure:
        pulse.call(copy.deepcopy(prepulse))
        for ind,pauli in enumerate(pauli_string):
            if(pauli=='X'):
                pulse.u2(0, np.pi, ind)
            if(pauli=='Y'):
                pulse.u2(0, np.pi/2, ind)
        for qubit in range(n_qubit):
            pulse.barrier(qubit)
        pulse.measure(range(n_qubit))
    return pulse_measure

def n_one(bitstring, key):
    results = 0
    for ind,b in enumerate(reversed(bitstring)):
        if((b=='1')&(key[ind]!='I')):
            results+=1
    return results

def expectation_value(counts,shots,key):
    results = 0
    for bitstring in counts:
        if(n_one(bitstring, key)%2==1):
            results -= counts[bitstring]/shots
        else:
            results += counts[bitstring]/shots
    return results

'''def run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot):
    results = pulse_backend.run(meas_pulse).result()
    counts = results.get_counts()
    expectation = expectation_value(counts,n_shot,key)
    return expectation'''
def run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot):
    # Misura il tempo di esecuzione della funzione run
    start_time_run = time.time()
    job = pulse_backend.run(meas_pulse, shots=n_shot)
    #job = backend.run(meas_pulse)
    run_time = time.time() - start_time_run

    # Monitora il job
    #job_monitor(job)
    
    # Misura il tempo di esecuzione per ottenere i risultati
    start_time_result = time.time()
    counts = job.result().get_counts()
    result_time = time.time() - start_time_result
    
    # Calcola l'aspettativa
    start_time_expectation = time.time()
    expectation = expectation_value(counts, n_shot, key)
    expectation_time = time.time() - start_time_expectation
    print('Tempo di esecuzione:',result_time)
    return expectation

def gen_LC_vqe(parameters):
    lb = np.zeros(parameters)
    ub = np.ones(parameters)
    LC = (LinearConstraint(np.eye(parameters),lb,ub,keep_feasible=False))
    return LC

In [ ]:
def vqe_one(prepulse,n_qubit,n_shot,pulse_backend, backend,key,value):
    all_Is = True
    for key_ele in key:
        if(key_ele!='I'):
            all_Is = False
    if(all_Is):
        return value
    meas_pulse = measurement_pauli(prepulse=prepulse, pauli_string=key, backend=backend, n_qubit=n_qubit)
    return value*run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot)

In [ ]:
def vqe(params,pauli_dict,pulse_backend, backend,n_qubit,n_shot):
    print("params in def chemistry in vqe.py: ", params)
    # assert(len(params)%2==0)
    width_len = int(len(params)-1*(n_qubit-1))
    split_ind = int(width_len/3)
    amp = np.array(params[:split_ind])
    angle = np.array(params[split_ind:width_len])*np.pi*2
    width_1 = (np.array(params[width_len:]))
    num_items = (1024 - 256) // 16 + 1
    width_norm = (width_1 - 256) / (1024 - 256)
    width_norm = np.clip(width_norm, 0, 1)
    width = (np.round(width_norm * (num_items - 1)) * 16 + 256).astype(int) 
    amp = amp.tolist()
    angle = angle.tolist()
    width = width.tolist()
    keys = [key for key in pauli_dict]
    values = [pauli_dict[key] for key in pauli_dict]
    expect_values = []

    for key, value in zip(keys, values):
        prepulse = HE_pulse(backend, amp, angle, width)
        expect = vqe_one(prepulse, n_qubit, n_shot, pulse_backend, backend, key, value)
        expect_values.append(expect)
    print("E for cur_iter: ",sum(expect_values))
    return sum(expect_values)

In [ ]:
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_aer.primitives import Estimator

def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

distances = np.arange(0.735, 0.736, 0.1)
exact_energies = []
pvqe_energies_1 = []
pvqe_energies_2 = []
iterazioni = []
errore = []

n_qubit = 2
parameters = 7
#np.random.seed(99999999)
LC = gen_LC_vqe(parameters)

n_shot = 1024
optimizer = 'COBYLA'
noiseless_estimator = Estimator(approximation=True)

for dist in distances:
    #params = np.zeros(parameters)
    '''params = [4.10768442e-03 , 4.38394468e-02,  5.41226278e-03,  1.83079826e-02,
  4.95038744e-03,  5.18470248e-03, -3.83842073e-18]'''
    '''params = [5.77602736e-02,  1.29297911e-01,  1.76817691e-17, -3.46944695e-18,
  1.28949827e-01,  0.00000000e+00,  0.00000000e+00]'''
    params = [-1.25000000e-02,  7.51377221e-02,  9.73794777e-02, -1.28501362e-18,
  4.68220056e-20, -1.13712956e-19,  1.23569996e-19] #nazca ca
    '''params = [-6.20864314e-20,  7.23188689e-02,  1.19644184e-01, -1.04083409e-17,
  5.29979964e-04, -2.51702279e-21,  1.87789325e-20]''' #osaka ca
    (qubit_op,my_dict,problem,REPULSION_ENERGYproblem) = get_qubit_op(dist)
    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies[0].real)
    print('distanza:', dist)
    vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                       method=optimizer, constraints=LC,options={'rhobeg':0.1,'maxiter':0})
    pvqe_energies_1.append(vqe_res.fun+ REPULSION_ENERGYproblem)
    err = abs(12/100 * vqe_res.fun)
    errore.append(err)
    print('distanza:',dist)
    print('pulse_VQE:',vqe_res.fun + REPULSION_ENERGYproblem,'+/-',err)
    print('esatto:',result.total_energies[0].real)

'''for dist in distances:
    #params = np.zeros(parameters)
    params = [-6.20864314e-20,  7.23188689e-02,  1.19644184e-01, -1.04083409e-17,
  5.29979964e-04, -2.51702279e-21,  1.87789325e-20]
    (qubit_op,my_dict,problem,REPULSION_ENERGYproblem) = get_qubit_op(dist)
    #result = exact_solver(qubit_op, problem)
    #exact_energies.append(result.total_energies[0].real)
    #print('distanza:', dist)
    vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                       method=optimizer, constraints=LC,options={'rhobeg':0.1,'maxiter':0})
    pvqe_energies_2.append(vqe_res.fun+ REPULSION_ENERGYproblem)
    err = abs(12/100 * vqe_res.fun)
    errore.append(err)
    print('distanza:',dist)
    print('pulse_VQE:',vqe_res.fun + REPULSION_ENERGYproblem,'+/-',err)
    #print('esatto:',result.total_energies[0].real)
'''

In [ ]:
import matplotlib.pyplot as plt
plt.plot(distances, pvqe_energies_1, marker = "*", linewidth=0.5, label="ibm_nazca")
#plt.plot(distances, pvqe_energies_2, marker = "*", linewidth=0.5, label="ibm_osaka")
#plt.plot(distances, pvqe_energies_3, marker = "*", linewidth=0.5, label="pulse ansatz VQE Energy")
#plt.plot(distances, pvqe_energies_4, marker = "*", linewidth=2.0, label="pulse ansatz VQE Energy")
plt.plot(distances, exact_energies, marker = "o", linewidth=2.0, label="Energy")
plt.xlabel("Distanza(Angstrom)")
plt.ylabel("Energia")
plt.legend()
#plt.title('Energia vs Distanza H2 (2qubit-FakeManila)')
output_path = 'EnergyVSdistH2(2qubit-FakeManila).png'
plt.savefig(output_path)
plt.show()

In [ ]:
from matplotlib import cbook
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import pandas as pd


fig = plt.figure(figsize=(12, 6))
xmin1, xmax1, ymin1, ymax1=0,2,-2,6
xmin2, xmax2, ymin2, ymax2=0.15,1.0,-1.5,0.5

# Create first axes, the top-left plot with green plot
sub1 = fig.add_subplot(1,2,1) # two rows, two columns, fist cell
sub1.set_xlim(xmin1, xmax1)
sub1.plot(distances, exact_energies, label="Exact Energy", linewidth=1)
sub1.plot(distances, pvqe_energies_1, label="VQE 65 iter",linewidth=1)
sub1.plot(distances, pvqe_energies_2, label="VQE 26 iter",linewidth=1)

sub1.set_ylabel('Energia', labelpad = 15)
sub1.set_xlabel('Distanza (Angstrom)', labelpad = 15)
sub1.legend()

sub2 = fig.add_subplot(1,2,2) # two rows, two columns, second cell
sub2.plot(distances, exact_energies, label="Exact Energy", linewidth=1)
sub2.plot(distances, pvqe_energies_1, label="VQE 65 iter",linewidth=1)
sub2.plot(distances, pvqe_energies_2, label="VQE 26 iter",linewidth=1)
sub2.set_xlim(xmin2, xmax2)
sub2.set_ylim(ymin2, ymax2)

x_fill=[xmin2,xmin2,xmax2,xmax2]
y_fill=[ymin2,ymax2,ymax2,ymin2]
sub1.fill(x_fill,y_fill,color='yellow',alpha=0.6,label='highlight')

con1 = ConnectionPatch(xyA=(xmax2, ymin2), coordsA=sub1.transData, 
                       xyB=(xmin2, ymin2), coordsB=sub2.transData, color = 'black')
fig.add_artist(con1)
con2 = ConnectionPatch(xyA=(xmax2, ymax2), coordsA=sub1.transData, 
                       xyB=(xmin2, ymax2), coordsB=sub2.transData, color = 'black')
fig.add_artist(con2)
#fig.suptitle('Energia vs Distanza H2 (2qubit-ibm_nazca)', fontsize=16)

plt.savefig('EnergyvsDistNAZCA.png')

plt.show()